In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
import matplotlib.pyplot as plt
import networkx as nx
import os

def create_cyclic_visualization(filename="cyclic_viz.png"):
    """
    Generates a simple Cyclic Graph diagram and saves it as an image.
    """
    G = nx.DiGraph()
    nodes = ["Plan", "Act", "Evaluate", "Learn"]
    G.add_edges_from([
        ("Plan", "Act"),
        ("Act", "Evaluate"),
        ("Evaluate", "Learn"),
        ("Learn", "Plan"),  # The Cycle
        ("Evaluate", "End") # The Exit
    ])

    pos = nx.circular_layout(G)

    plt.figure(figsize=(6, 6))

    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='skyblue', edgecolors='black')

    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=12, font_family="sans-serif", font_weight="bold")

    # Draw edges
    nx.draw_networkx_edges(G, pos, width=2, arrowsize=20, arrowstyle='->', connectionstyle="arc3,rad=0.1")

    plt.title("Cyclic State Graph Pattern", fontsize=15, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    return filename

def create_slide():
    # 1. Create the Visualization first
    img_path = create_cyclic_visualization()

    # 2. Create Presentation
    prs = Presentation()

    # Use a blank layout (6) to have full control, or Title (0) / Title & Content (1)
    # We will use a blank layout to arrange text on left and image on right manually
    slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(slide_layout)

    # --- ADD TITLE ---
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.4), Inches(9), Inches(1))
    title_frame = title_box.text_frame
    title_p = title_frame.add_paragraph()
    title_p.text = "Why LangGraph? The Need for Cyclic State Machines"
    title_p.font.size = Pt(32)
    title_p.font.bold = True
    title_p.font.name = 'Arial'

    # --- ADD CONTENT (Left Side) ---
    # Define text box dimensions
    content_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.5), Inches(5.5), Inches(5.5))
    tf = content_box.text_frame
    tf.word_wrap = True

    # Helper to add bullet points
    def add_bullet(text_frame, text, level=0, bold=False, size=16):
        p = text_frame.add_paragraph()
        p.text = text
        p.level = level
        p.font.size = Pt(size)
        p.font.name = 'Arial'
        if bold:
            p.font.bold = True
        # Add a little spacing
        p.space_before = Pt(6)
        return p

    # Section 1
    add_bullet(tf, "1. The Limitation of Linear Chains (DAGs)", level=0, bold=True, size=18)
    add_bullet(tf, "Rigid Execution: Standard LLM chains follow a strict Input -> Step A -> Step B pipeline.", level=1)
    add_bullet(tf, "Fragility: If one step fails, the entire chain breaks.", level=1)
    add_bullet(tf, "No \"Retry\" Logic: Cannot naturally loop back to fix mistakes without restarting.", level=1)

    # Section 2
    add_bullet(tf, "2. The Solution: Cyclic State Graphs", level=0, bold=True, size=18)
    add_bullet(tf, "Loops & Cycles: Agents can Plan -> Act -> Fail -> Learn -> Retry.", level=1)
    add_bullet(tf, "Persistence (State): Shared memory persists across loops, remembering prior failures.", level=1)
    add_bullet(tf, "Engineering Control: Replace prompt magic with code (Conditional Edges).", level=1)

    # Key Takeaway
    p = tf.add_paragraph()
    p.text = "\nKey Takeaway: Real agents need to iterate. LangGraph provides the architecture for iteration."
    p.font.size = Pt(16)
    p.font.bold = True
    #p.font.color.rgb = 0x000000 # Black (or use RGBColor for custom)

    # --- ADD IMAGE (Right Side) ---
    # Adjust position to sit to the right of the text
    slide.shapes.add_picture(img_path, Inches(6.2), Inches(2.0), width=Inches(3.5))

    # --- SAVE ---
    output_filename = "LangGraph_Intro_Slide.pptx"
    prs.save(output_filename)
    print(f"Presentation saved as '{output_filename}'")

    # Cleanup the temporary image
    if os.path.exists(img_path):
        os.remove(img_path)

if __name__ == "__main__":
    create_slide()

In [ ]:
import operator
from typing import Annotated, List, Literal, TypedDict, Union
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

# --- 1. CONFIGURATION & SCHEMA ---

# A. Define the Structured Output Schema (The goal)
class WeatherReport(BaseModel):
    """The final structured response we want."""
    city: str = Field(description="The city mentioned")
    temperature: str = Field(description="The temperature found")
    advice: str = Field(description="Clothing advice based on weather")
    confidence: int = Field(description="Confidence score 1-10")

# B. Define the State
class AgentState(TypedDict):
    # 'operator.add' appends new messages to the history rather than overwriting
    messages: Annotated[List[BaseMessage], operator.add]

# --- 2. DEFINE TOOLS ---

@tool
def get_weather(city: str) -> str:
    """Fetches the current weather for a specific city."""
    # Mock data for demonstration
    if "warsaw" in city.lower():
        return "It is rainy and 12°C."
    elif "sf" in city.lower() or "san francisco" in city.lower():
        return "It is sunny and 22°C."
    return "Weather unknown."

tools = [get_weather]

# --- 3. DEFINE NODES ---

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Node 1: The Agent (Reasoning & Tool Selection)
def agent_node(state: AgentState):
    # Bind tools so the LLM knows it can use them
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# Node 2: Tool Execution (Prebuilt by LangGraph)
tool_node = ToolNode(tools)

# Node 3: Structured Formatter (The Final Step)
def formatter_node(state: AgentState):
    """Takes the conversation history and generates the final JSON."""
    structured_llm = llm.with_structured_output(WeatherReport)

    # We summarize the entire conversation into the strict schema
    prompt = "Review the conversation history and generate the final report."
    # We pass the history + the prompt
    chain = structured_llm

    # Invoke with the full message history
    report = chain.invoke(state["messages"] + [HumanMessage(content=prompt)])

    # Return as a special AIMessage or just print it
    # Here we return it as a message to store in history
    return {"messages": [AIMessage(content=f"FINAL_JSON: {report.json()}")]}

# --- 4. DEFINE EDGES (Logic) ---

def should_continue(state: AgentState):
    last_message = state["messages"][-1]

    # If the LLM called a tool, go to 'tools' node
    if last_message.tool_calls:
        return "tools"

    # If LLM is done (has text), go to 'formatter' to structure it
    return "formatter"

# --- 5. BUILD GRAPH ---

workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)
workflow.add_node("formatter", formatter_node)

# Set Entry
workflow.set_entry_point("agent")

# Add Edges
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "formatter": "formatter"
    }
)

workflow.add_edge("tools", "agent")     # Loop back: Agent -> Tools -> Agent
workflow.add_edge("formatter", END)     # Finish after formatting

# --- 6. COMPILE WITH MEMORY ---

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# --- 7. RUN DEMONSTRATION ---

# Config acts as the session ID
config = {"configurable": {"thread_id": "session_001"}}

print("--- Turn 1: User asks for weather (Triggers Tool) ---")
inputs = {"messages": [HumanMessage(content="Check the weather in Warsaw.")]}

for event in app.stream(inputs, config=config):
    for node, values in event.items():
        # Print only the new messages added by the node
        msg = values["messages"][-1]
        print(f"📍 Node '{node}': {msg.content if isinstance(msg.content, str) else 'Tool Call'}")

print("\n" + "="*40 + "\n")

print("--- Turn 2: User asks follow up (Triggers Memory + Structured Output) ---")
# Note: We don't mention Warsaw again, but the agent remembers it from thread_id
inputs_2 = {"messages": [HumanMessage(content="Is that cold? Generate the report.")]}

for event in app.stream(inputs_2, config=config):
    for node, values in event.items():
        msg = values["messages"][-1]
        print(f"📍 Node '{node}': {msg.content}")

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
#from langchain_core.chains import LLMChain

# Setup LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# 1. Define the CoT Prompt [cite: 34]
# The PDF example uses a math word problem about mangoes
template = """
I went to the market and bought 20 mangoes.
I gave 5 apples to the Ram and 3 to the Househelp.
I then went and bought 12 more mangoes and ate 2.
How many mangoes did I remain with?

Let's think step by step.
"""

prompt = PromptTemplate.from_template(template)

# 2. Run the Chain
chain = prompt | llm

print("--- Chain of Thought Trace ---")
response = chain.invoke({})
print(response.content)

# Expected Trace (from PDF [cite: 43-49]):
# "Initial mangoes bought: You started with 20 mangoes.
#  ...
#  Now, you have 12 (initial) + 12 (additional) = 24 mangoes.
#  Mangoes eaten: You ate 2 mangoes. Subtract 2 from the total: 24-2=22.
#  So, after these steps, you have 22 mangoes remaining."

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
# Note: Requires langchain-experimental package [cite: 9]
from langchain_experimental.smart_llm import SmartLLMChain

# Setup LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# 1. Define the 'Hard Question' Prompt [cite: 260]
hard_question = "I have a 12 liter jug, a 4 liter jug and a 3 liter jug. I want to measure 6 liters. How do I do it?"
prompt = PromptTemplate.from_template(hard_question)

# 2. Initialize SmartLLMChain (Tree of Thought) [cite: 263]
# n_ideas=3 generates 3 branches of thought
chain = SmartLLMChain(
    llm=llm,
    prompt=prompt,
    n_ideas=3,
    verbose=True
)

print("\n--- Tree of Thought (SmartLLM) Trace ---")
chain.run({})

# Expected Trace (from PDF [cite: 280-468]):
# Idea 1: Step 1: Fill 12L... Step 2: Pour into 4L...
# Idea 2: ...
# Idea 3: ...
# Critique: Idea 1 seems correct... Idea 2 is not feasible because...
# Final Answer: (Selects best path)


--- Tree of Thought (SmartLLM) Trace ---


> Entering new SmartLLMChain chain...
Prompt after formatting:
I have a 12 liter jug, a 4 liter jug and a 3 liter jug. I want to measure 6 liters. How do I do it?


C:\Users\marci\AppData\Local\Temp\ipykernel_19744\2082568738.py:23: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  chain.run({})


APIConnectionError: Connection error.